## Загрузка данных и инициализация модели

In [ ]:
# https://drive.google.com/file/d/1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG/view?usp=sharing # ссылка для ручной загрузки данных

In [ ]:
!pip install gdown

In [ ]:
!gdown 1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG && tar -xf kaggle.tar

In [ ]:
!ls -l kaggle/wav | wc -l

Для базового решения задания воспользуемся предобученной моделью Whisper от OpenAI. Для этого обратимся к платформе [HuggingFace](https://huggingface.co/collections/openai/whisper-release-6501bba2cf999715fd953013)

In [ ]:
import os
import soundfile as sf
import torch
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import string

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from torch.nn.parallel import DataParallel
device = torch.device("cuda:0")
model_name = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_name)
processor.feature_extractor.return_attention_mask = True
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)


## Baseline подход к инференсу модели

In [ ]:
def batch_inference(model, processor, path_to_wavs, batch_size, sampling_rate=16000):
    results = {}
    wav_files = os.listdir(path_to_wavs)

    forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe") # generates task specific special tokens
    results = {}
    for i in tqdm(range(0, len(wav_files), batch_size), total=np.ceil(len(wav_files) / batch_size)):
        audio_paths = wav_files[i : i + batch_size]

        batch = []

        for path in audio_paths:
            audio, _ = sf.read(os.path.join(path_to_wavs, path))
            batch.append(audio)
        inputs = processor(batch, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

        x, x_masks = inputs["input_features"].to(device), inputs["attention_mask"].to(device)

        with torch.no_grad():
            model.eval()
            output_ids = model.generate(x, forced_decoder_ids=forced_decoder_ids, attention_mask=x_masks,num_beams = 2, num_return_sequences = 2, repetition_penalty=1.5, return_dict = True)

        transcribtion = processor.batch_decode(output_ids, skip_special_tokens=True)
        transcribtion = iter(transcribtion)
        
        for path in audio_paths:
            for j in range(2):
                results.setdefault(path, []).append(next(transcribtion))


        
        
    return results

In [ ]:
path_to_wavs = "kaggle/wav"
batch_size = 3

In [ ]:
results = batch_inference(model=model, processor=processor, path_to_wavs=path_to_wavs, batch_size=batch_size)

In [ ]:
results 

## Отранжируем предложенные варианты с помощью BERT

In [ ]:
import torch

from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Определите функцию для оценки уверенности BERT
def evaluate_confidence(text):
    inputs = tokenizer.encode_plus(text, 
                                    add_special_tokens=True, 
                                    max_length=512, 
                                    return_attention_mask=True, 
                                    return_tensors='pt')
    outputs = model(inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))
    confidence = torch.nn.functional.softmax(outputs.last_hidden_state[:, 0, :], dim=1)
    return confidence.max().item()

# Пройдите по словарю и оставьте только одно предложение в каждом ключе
for key, values in results.items():
    confidences = [evaluate_confidence(value) for value in values]
    best_index = confidences.index(max(confidences))
    results[key] = [values[best_index]]


In [ ]:
results

In [ ]:
!pip install num2words

In [ ]:
import num2words

# Функция для замены цифр на слова
def replace_numbers_with_words(text):
    import re
    numbers = re.findall(r'\d+', text)
    for number in numbers:
        text = text.replace(number, num2words.num2words(int(number), lang='ru'))
    return text

# Пройдите по словарю и замените цифры на слова
for key, values in results.items():
    new_values = []
    for value in values:
        new_value = replace_numbers_with_words(value)
        new_values.append(new_value)
    results[key] = new_values


In [ ]:
results

## Sample submission
Подготовим пример для загрузки на платформу

In [ ]:
import string

def dummy_postprocessing(data):
    for filename, hypo in data.items():
        hypo = hypo[0].strip()  # берем первый элемент списка
        hypo = hypo.translate(str.maketrans('', '', string.punctuation))
        # hypo = hypo.replace('[', '').replace(']', '')  # удаляем скобки
        hypo = hypo.lower()
        if not hypo:
            data[filename] = "-"
        else:
            data[filename] = hypo  # сохраняем результат в виде списка
    return data

clean_data = dummy_postprocessing(results)


In [ ]:
sample_submission = pd.read_csv("kaggle/sample_submission.csv")

In [ ]:
sample_submission.head(20)

In [ ]:
sample_submission["hypo"] = sample_submission["filename"].apply(lambda x: clean_data[x])

In [ ]:
sample_submission.insert(0, "id", sample_submission.index)

In [ ]:
sample_submission.head(10)

In [ ]:
sample_submission.to_csv("baseline.csv", index=False)

## Идеи по улучшению предсказаний:
* Всегда ли LAS архитектура работает на тишине корректно?
* Как насчет нормализации текста? (19.02 - девятнадцатое февраля, 20 лет - двадцать лет)
* Может быть попробовать рескоринг? (возвращаем больше, чем топ-1 гипотезу из бима)
* Может быть взять модель побольше?
